### Imports

In [ ]:
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:6000'

In [ ]:
!set 'PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:7000'
!set 'CUDA_LAUNCH_BLOCKING=1'

In [1]:
%load_ext autoreload
%autoreload 2

from IPython.display import clear_output

import os
import re
import sys
import json
from typing import List
from datetime import datetime
from tqdm import tqdm
from functools import partial

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import pytorch_lightning as pl
from transformers import BertTokenizerFast, XLMRobertaTokenizerFast, AutoTokenizer
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset

from src.manipulation_helpers.si import SILitModel
from src.manipulation_helpers.st import STLitModel
from src.manipulation_helpers.data_preparation import markup_conll, encode_tags, Markup, \
                                                      read_markup, create_span_targeting_data, \
                                                      SpanTargetingDataset


In [2]:
def custom_split(tagged_text):
    text = []
    tags = []
    word = ''
    prev_tag = 0
    for letter, tag in tagged_text:
        if letter == ' ' or letter == '\n':
            if word == '':
                continue
                
            text.append(word)
            tags.append(prev_tag)
            word = ''
            prev_tag = 0
        else:
            word += letter
            prev_tag = tag
    return text, tags

def create_bin_span(text, bounds):
    span = [0 for _ in range(bounds[0][0])]
    for i, bound in enumerate(bounds):
        if i == len(bounds) - 1:
            span += [1 for _ in range(bound[1] - bound[0])]
            span += [0 for _ in range(len(text) - bound[1])]
        else:
            span += [1 for _ in range(bound[1] - bound[0])]
            span += [0 for _ in range(bounds[i + 1][0] - bound[1])]
    return [(letter, tag) for letter, tag in zip(text, span)]

def post_proc(span):
    for i in range(len(span) - 1):
        if span[i] == 0 and span[i + 1] == 1:
            span[i + 1] = 2
    return span 

In [3]:
PATH_TO_MARKUP = "data/markup_union_matched.json"
PATH_TO_ENG_DATA = "data/data/protechn_corpus_eval/train"
MODEL_NAME = "xlm-roberta-base"# "xlm-roberta-base""DeepPavlov/rubert-base-cased"

## Data processing

In [4]:
eng_texts = []
eng_spans = []

for filename in os.listdir(PATH_TO_ENG_DATA):
    if filename.endswith('.tsv'):
        idx = re.search('\d+', filename).group(0)
        text = open(PATH_TO_ENG_DATA + f'/article{idx}.txt', 'rb').read().decode('utf-8')
        try:
            spans = pd.read_table(PATH_TO_ENG_DATA + f'/{filename}', header=None)
        except:
            continue
        bounds = []
        for i in range(len(spans)):
            bounds.append((spans[2].iloc[i], spans[3].iloc[i]+1))
        tagged_text = create_bin_span(text, bounds)
        text, tags = custom_split(tagged_text)
        eng_texts.append(text)
        eng_spans.append(tags)

eng_spans = list(map(post_proc, eng_spans))

In [5]:
markup = pd.read_json(PATH_TO_MARKUP, lines=True)
print("Всего разметок:", len(markup))

Всего разметок: 1686


In [6]:
texts = []
markups = []
for index, row in markup.iterrows():
    text, row_markup = markup_conll(row["input_input"], 
                                    row["output_result"], 
                                    row["input_entitiesdata"])

    texts.append(text)
    markups.append(row_markup)
clear_output()

In [7]:
unique_tags = {x.manipulation_class for m in markups for x in m}
tag2id = {tag: idx for idx, tag in enumerate(unique_tags)}
id2tag = {idx: tag for tag, idx in tag2id.items()}

In [8]:
bio2id = {
    'O': 0,
    'I': 1,
    'B': 2,
}
id2bio = {
    idx : tag for tag, idx in bio2id.items()
}

In [9]:
train_texts, val_texts, train_markups, val_markups, train_ids, val_ids = \
train_test_split(texts, markups, range(len(texts)), test_size=.2, random_state=42)

### Dataset & tokenization

In [10]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME) #BertTokenizerFast

encode = partial(
    tokenizer,
    add_special_tokens=False, 
    is_split_into_words=True, 
    padding=True, 
    truncation=True, 
    max_length=512)

encoded_sep_token = encode([tokenizer.sep_token])['input_ids'][0]
encoded_cls_token = encode([tokenizer.cls_token])['input_ids'][0]

train_encodings = encode(train_texts)
val_encodings = encode(val_texts)
#eng_encodings = encode(eng_texts)

In [11]:
train_labels = encode_tags(train_markups, train_encodings, "manipulation_class", tag2id=tag2id)
val_labels = encode_tags(val_markups, val_encodings, "manipulation_class", tag2id=tag2id)

train_entities = encode_tags(train_markups, train_encodings, "entity_id")
val_entities = encode_tags(val_markups, val_encodings, "entity_id")

train_manipulation_targets = encode_tags(train_markups, train_encodings, "manipulation_target")
val_manipulation_targets = encode_tags(val_markups, val_encodings, "manipulation_target")

train_spans = encode_tags(train_markups, train_encodings, "bio_span", tag2id=bio2id)
val_spans = encode_tags(val_markups, val_encodings, "bio_span", tag2id=bio2id)
#eng_spans = encode_tags(eng_spans, eng_encodings, 'eng')

# 1. Span Identification

In [ ]:
train_x, train_y = train_encodings['input_ids'], train_spans
#eng_x, eng_y = eng_encodings['input_ids'], eng_spans
#train_x = train_x + eng_x
#train_y = train_y + eng_y
val_x, val_y = val_encodings['input_ids'], val_spans

In [ ]:
assert (len(train_x) == len(train_y)) & (len(val_x) == len(val_y)), "Чето не так"

In [ ]:
print(f'Новостей в обучении: {len(train_y)}')
print(f'Новостей в валидации: {len(val_y)}')

In [ ]:
train_x, train_y = torch.tensor(train_x), torch.tensor(train_y)
val_x, val_y = torch.tensor(val_x), torch.tensor(val_y)

train_dataset = TensorDataset(train_x, train_y)
val_dataset = TensorDataset(val_x, val_y)

train_loader = DataLoader(train_dataset, batch_size=8)
val_loader = DataLoader(val_dataset, batch_size=8)

In [ ]:
(val_y == 0).sum() / (val_y == 1).sum(), (train_y == 0).sum() / (train_y == 1).sum()

### Pytorch-Lightning Model

In [ ]:
config = {
    'bert_model_name': MODEL_NAME,
    'optimizer': torch.optim.AdamW, 
    'lr': 1e-5, 
    'freeze_bert': True,
    'loss_function': torch.nn.CrossEntropyLoss(weight=torch.tensor([1., 2., 2.]), ignore_index=-100),
    'output_size': 3,
    'scheduler': partial(torch.optim.lr_scheduler.ExponentialLR, gamma=0.99),
    'freeze_all': True
}
model = SILitModel(**config)
trainer = pl.Trainer(max_epochs=30)
trainer.fit(model, train_loader, val_loader)

In [ ]:
trainer = pl.Trainer(max_epochs=10)
trainer.fit(model, train_loader, val_loader)

In [ ]:
trainer = pl.Trainer(max_epochs=10)
trainer.fit(model, train_loader, val_loader)

In [ ]:
x, y = next(iter(train_loader))
attention_mask = (y != -100)

In [ ]:
pytorch_total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

In [ ]:
pytorch_total_params

# 2. Span Targetting

In [12]:
train_full_texts, train_entities, train_spans, train_labels = \
create_span_targeting_data(train_encodings, train_entities, train_manipulation_targets, train_spans, encoded_cls_token)

val_full_texts, val_entities, val_spans, val_labels = \
create_span_targeting_data(val_encodings, val_entities, val_manipulation_targets, val_spans, encoded_cls_token)

% of positive class 0.457238059977786
% of positive class 0.39741750358680056


In [13]:
assert (np.array(list(map(len, train_full_texts))) == 512).all()
assert (np.array(list(map(len, train_entities))) == 65).all()
assert (np.array(list(map(len, train_spans))) == 257).all()

assert (np.array(list(map(len, val_full_texts))) == 512).all()
assert (np.array(list(map(len, val_entities))) == 65).all()
assert (np.array(list(map(len, val_spans))) == 257).all()

In [14]:
train_loader = DataLoader(SpanTargetingDataset(
    train_full_texts, train_entities, train_spans, train_labels
), batch_size=4, num_workers=2)

val_loader = DataLoader(SpanTargetingDataset(
    val_full_texts, val_entities, val_spans, val_labels
), batch_size=4, num_workers=2)

In [ ]:
train_labels

### Pytorch-Lightning Model

In [15]:
config = {
    'bert_model_name': MODEL_NAME,
    'optimizer': torch.optim.AdamW, 
    'lr': 1e-5, 
    'freeze_bert': True,
    'loss_function': torch.nn.CrossEntropyLoss(),
    'output_size': 2,
    'scheduler': partial(torch.optim.lr_scheduler.ExponentialLR, gamma=0.99),
    'freeze_all': True,
    'mode': 'cross'
}
model = STLitModel(**config)
trainer = pl.Trainer(max_epochs=20)
trainer.fit(model, train_loader, val_loader)

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaModel: ['lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type             | Params
-------------------------------------------

Sanity Checking: 0it [00:00, ?it/s]

E:\anaconda\envs\manipulation_env_v4\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:430: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
E:\anaconda\envs\manipulation_env_v4\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:430: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

E:\anaconda\envs\manipulation_env_v4\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Иван\OneDrive\Рабочий стол\Lukyanenko-BS-Thesis\src\manipulation_helpers\st.py:117: RuntimeWarning: invalid value encountered in scalar divide
  f1 = np.nan_to_num(2 * precision * recall / (precision + recall))
E:\anaconda\envs\manipulation_env_v4\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
E:\anaconda\envs\manipulation_env_v4\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due t

E:\anaconda\envs\manipulation_env_v4\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
E:\anaconda\envs\manipulation_env_v4\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Иван\OneDrive\Рабочий стол\Lukyanenko-BS-Thesis\src\manipulation_helpers\st.py:117: RuntimeWarning: invalid value encountered in scalar divide
  f1 = np.nan_to_num(2 * precision * recall / (precision + recall))
E:\anaconda\envs\manipulation_env_v4\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no 

E:\anaconda\envs\manipulation_env_v4\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Иван\OneDrive\Рабочий стол\Lukyanenko-BS-Thesis\src\manipulation_helpers\st.py:117: RuntimeWarning: invalid value encountered in scalar divide
  f1 = np.nan_to_num(2 * precision * recall / (precision + recall))
E:\anaconda\envs\manipulation_env_v4\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Иван\OneDrive\Рабочий стол\Lukyanenko-BS-Thesis\src\manipulation_helpers\st.py:117: RuntimeWarning: invalid value encountered in scalar divide
  f1 = np.nan_to

E:\anaconda\envs\manipulation_env_v4\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Иван\OneDrive\Рабочий стол\Lukyanenko-BS-Thesis\src\manipulation_helpers\st.py:117: RuntimeWarning: invalid value encountered in scalar divide
  f1 = np.nan_to_num(2 * precision * recall / (precision + recall))
E:\anaconda\envs\manipulation_env_v4\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Иван\OneDrive\Рабочий стол\Lukyanenko-BS-Thesis\src\manipulation_helpers\st.py:117: RuntimeWarning: invalid value encountered in scalar divide
  f1 = np.nan_to

E:\anaconda\envs\manipulation_env_v4\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
E:\anaconda\envs\manipulation_env_v4\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Иван\OneDrive\Рабочий стол\Lukyanenko-BS-Thesis\src\manipulation_helpers\st.py:117: RuntimeWarning: invalid value encountered in scalar divide
  f1 = np.nan_to_num(2 * precision * recall / (precision + recall))
E:\anaconda\envs\manipulation_env_v4\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no 

In [ ]:
model.head.weight.grad

In [ ]:
it = iter(val_loader)

In [ ]:
batch = next(it)

In [ ]:
batch[3]

### Evaluation

In [ ]:
config = {
    'bert_model_name': MODEL_NAME,
    'optimizer': torch.optim.AdamW, 
    'lr': 1e-6, 
    'freeze_bert': False,
    'loss_function': torch.nn.CrossEntropyLoss(weight=torch.tensor([1., 8.]), ignore_index=-100), 
    'output_size': 2
}
model = ManipulationTargetLightningModel(**config)
model.load_state_dict(torch.load('lightning_logs\\version_33\\checkpoints\\epoch=79-step=18240.ckpt')['state_dict'])

In [ ]:
predictions

In [ ]:
model.to('cuda')
map_val_id2scores = {'id': [], 'acc': [], 'jac': [], 'p': [], 'r': []}
for val_id, sample_ids in tqdm(map_val_ids2sample_ids.items()):
    
    accuracy_scores = []
    jac_scores = []
    r_scores = []
    p_scores = []
    
    for x, y in zip(*val_dataset[sample_ids]):
        mask = torch.where(y == -100)[0]
        attention_mask = (y != -100)
        start = mask[0]
        if mask.shape[0] <= 1:
            end = 512
        else:
            end = mask[1]
            
        prediction = model(x.unsqueeze(0).to('cuda'), attention_mask.unsqueeze(0).to('cuda'))[0]
        prediction = prediction.argmax(-1)
        
        accuracy_scores.append((prediction[start:end].cpu() == y[start:end]).sum() / (end-start))
        
        jac_scores.append((
            (prediction[start:end].cpu() == 1) & (y[start:end] == 1)).sum() / \
            ((y[start:end] == 1) | (prediction[start:end].cpu() == 1)).cpu().sum())
        
        r_scores.append((
            (prediction[start:end].cpu() == 1) & (y[start:end] == 1)).sum() / \
        
        p_scores.append((
            (prediction[start:end].cpu() == 1) & (y[start:end] == 1)).sum() / \
            (y[start:end].cpu() == 1).cpu().sum())
        
        if (prediction == 1.).sum() == 0 and (y == 1.).sum() == 0:
            jac_scores[-1] = 1
            accuracy_scores[-1] = 1
            p_scores[-1] = 1
            r_scores[-1] = 1
            
    map_val_id2scores['id'].append(val_id)
    map_val_id2scores['acc'].append(np.mean(accuracy_scores))
    map_val_id2scores['jac'].append(np.mean(jac_scores))
    map_val_id2scores['p'].append(np.mean(p_scores))
    map_val_id2scores['r'].append(np.mean(r_scores))
    
    if np.mean(p_scores).item() > 0.5:
        print(val_id)
        print(jac_scores)
        print(start, end)
        print(prediction[start:end])
        print(y[start:end])
        print(f'Text: {tokenizer.decode(x[start:end])}')
        print(f'Predicted: {tokenizer.decode(x[torch.where(prediction[start:end].cpu() == 1)])}')
        print(f'True: {tokenizer.decode(x[torch.where(y[start:end].cpu() == 1)])}')

In [ ]:
metrics = pd.DataFrame.from_dict(map_val_id2scores)

In [ ]:
metrics['target'] = metrics.apply(lambda row: 1 if row['jac'] > 0 else 0, axis=1)
metrics

In [ ]:
metrics[metrics.acc != 1].mean()

In [ ]:
metrics[metrics.jac != 1].jac.mean()

In [ ]:
markup.loc[metrics[metrics.jac != 1].id].rel_jac.mean()

In [ ]:
val.rel_jac.fillna(0).mean()

In [ ]:
metrics.target.mean() / (val.rel_target.fillna(0) > 0).astype(int).mean()

In [ ]:
metrics.jac.mean() / (val.rel_jac.fillna(0) > 0).astype(int).mean()

In [ ]:
(val.rel_target.fillna(0)).mean()

In [ ]:
(val.rel_target.fillna(0) > 0).astype(int).mean()

In [ ]:
mask = torch.where(y == -100)[0]
start = mask[0]
if mask.shape[0] <= 1:
    end = 512
else:
    end = mask[1]

In [ ]:
(prediction[start:end].cpu() == 1) == (y[start:end] == 1)).sum()

In [ ]:
torch.where(y == -100)

In [ ]:
y[torch.where(y == -100)[0][0] + 1:]

In [ ]:
y

In [ ]:
markup.loc[val_ids]

In [ ]:
(y != -100)

In [ ]:
for x, y in val_dataset[101]:
    attention_mask = (y != 100)
    preds = model(x.unsqueeze(0), attention_mask.unsqueeze(0))
    predictions = preds.argmax(-1)
    
    for token_ids, ground_truth, prediction in zip(x, y, predictions):
        print()
        print(f'Text: {tokenizer.decode(token_ids).split("[SEP]")[1].split("[PAD]")[0]}')
        print()
        print(f'Entity: {tokenizer.decode(token_ids).split("[SEP]")[0]}')
        print()
        print(f'Manipulation span: {tokenizer.decode(token_ids[np.where(np.array(ground_truth) == 1)])}')
        print()
        print(f'Predicted span: {tokenizer.decode(token_ids[np.where(np.array(prediction.cpu()) == 1)])}')
        print()
        print('############################################################')
        break

In [ ]:
token_ids, ground_truth = val_dataset[101]
attention_mask = (ground_truth != 100)
preds = model(token_ids.unsqueeze(0).to('cuda'), attention_mask.unsqueeze(0).to('cuda'))[0]
predictions = preds.argmax(-1)
mask = torch.where(ground_truth == -100)[0]
start = mask[0]
if mask.shape[0] <= 1:
    end = 512
else:
    end = mask[1]


print(((predictions[start:end].cpu() == 1) & (ground_truth[start:end] == 1)).sum() / \
            ((ground_truth[start:end] == 1) | (predictions[start:end].cpu() == 1)).cpu().sum())
    
print(predictions)

print()
print(f'Text: {tokenizer.decode(token_ids).split("[SEP]")[1].split("[PAD]")[0]}')
print()
print(f'Entity: {tokenizer.decode(token_ids).split("[SEP]")[0]}')
print()
print(f'Manipulation span: {tokenizer.decode(token_ids[np.where(np.array(ground_truth) == 1)])}')
print()
print(f'Predicted span: {tokenizer.decode(token_ids[np.where(np.array(predictions.cpu()) == 1)])}')
print()
print('############################################################')

In [ ]:
tokenizer.decode(token_ids)

In [ ]:
texts[529]

In [ ]:
|